# Scientific Computing: Diffusion Limited Aggregation and Reaction Diffusion

## Imports

In [ ]:
import numpy as np
from IPython.display import HTML

import src.solutions as solutions
import src.visualizations as visualizations